In [1]:
import pennylane as qml
import pennylane.numpy as np
import jax.numpy as jnp
from scipy.special import legendre
import jax
import optax

from time import time
import pickle

import matplotlib.pyplot as plt


import os, sys, argparse

parent = os.path.abspath('../src')
sys.path.insert(1, parent)

from perceptron import NativePerceptron



# Set to float64 precision and remove jax CPU/GPU warning
jax.config.update("jax_enable_x64", True)
jax.config.update("jax_platform_name", "cpu")

In [2]:
# setting up the problem
perceptron_qubits = 3
pulse_basis = 2
ts = jnp.array([1.0])
t = 1

dev = qml.device("default.qubit.jax", wires = perceptron_qubits)


perceptron =NativePerceptron(perceptron_qubits, pulse_basis, basis='gaussian', pulse_width=0.005, native_coupling=1)

H =  perceptron.H

H_obj, H_obj_spectrum = perceptron.get_1d_ising_hamiltonian(0.1)

# e_ground_state_exact = H_obj_spectrum[0]

print(f'Ising Model Hamiltonian:\nH = {H_obj}')
# print(f'Exact ground state energy: {e_ground_state_exact}')


Ising Model Hamiltonian:
H =   (0.1) [X0]
+ (0.1) [X1]
+ (0.1) [X2]
+ (1.0) [Z0 Z1]
+ (1.0) [Z1 Z2]


I0000 00:00:1706304169.774383       1 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.
/Users/jorgegarciaponce/Research/Yelin/PerceptronLossLandscapes/.venv/lib/python3.11/site-packages/pennylane/ops/functions/eigvals.py:108: UserWarning: For Hamiltonians, the eigenvalues will be computed numerically. This may be computationally intensive for a large number of wires. Consider using a sparse representation of the Hamiltonian with qml.SparseHamiltonian.
  warnings.warn(


In [3]:
param_vector = perceptron.get_random_parameter_vector(89889)

In [12]:
@qml.qnode(dev, interface="jax")

def get_expectation(perceptron, tp, tk, pauli_op_a, pauli_op_b, param_vector):

    param_list = perceptron.vector_to_hamiltonian_parameters(param_vector)

    qml.evolve(perceptron.H)(param_list, tp)

    U = qml.evolve(perceptron.H)(param_list, [tp, tk])

    return qml.expval((qml.adjoint(U)@pauli_op_a@U)@ pauli_op_b )


get_expectation(perceptron, 0.33, 0.66, qml.PauliX(1), qml.PauliY(2), param_vector)


/Users/jorgegarciaponce/Research/Yelin/PerceptronLossLandscapes/.venv/lib/python3.11/site-packages/pennylane/measurements/expval.py:56: UserWarning: Prod might not be hermitian.
  warnings.warn(f"{op.name} might not be hermitian.")
/Users/jorgegarciaponce/Research/Yelin/PerceptronLossLandscapes/.venv/lib/python3.11/site-packages/jax/_src/numpy/lax_numpy.py:2054: ComplexWarning: Casting complex values to real discards the imaginary part
  out = np.array(object, dtype=dtype, ndmin=ndmin, copy=False)  # type: ignore[arg-type]


Array(0.455465, dtype=float64)